# Project: Youtube Analyzer
## Authors: Karmehr Arora, Alfred Koh

### Project Overview
Data: Youtube API’s
GUI: Tkinter

Roles:

    Karmehr: Accessing Data from the API & organizing it

    Alfred: Analyzing the data & representing it with tkinter

Goal: 
Finding the top 10 trending basketball videos (or videos of a requested topic) in the last 24 hours
Look for the most (total likes + total comments + total shares) under tag = basketball in the last 24 hours

Limitations: 
Estimated limit of searches of 50 per day due to Youtube API usage restrictions


In [1]:
# import utilized libraries pertaining to data collection
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

In [7]:
# get access key to API
from getpass import getpass
API_KEY = getpass('Enter your API Key:')

Enter your API Key:········


In [8]:
# define the api service and version
api_service_name = 'youtube'
api_version = 'v3'

## Requesting User to choose topic
default to basketball as topic if user has error/opts out of entering topic

In [65]:
# request user for topic (default = 'basketball')
topic = 'basketball'
topic = input()

Formula 1


## Search youtube for videos with that tag

In [66]:
# access the youtube api client
youtube = googleapiclient.discovery.build(api_service_name, api_version, developerKey = API_KEY)

# make a request to the api to find videos under that topic
# note: cost of this operation is 100 units of the alloted 10,000 units per 24 hours
request = youtube.search().list(
        part="id,snippet",
        maxResults=50,
        q=topic
    )

In [67]:
# execute the query in the request
response = request.execute()

In [75]:
print("Search ID: ", response["items"][0]['id'])
print("\nSnippet Data:",response["items"][0]['snippet'])
videos = response["items"]
count = 1
for video in videos:
    if('videoId' in video['id']):
        print(count, ": https://www.youtube.com/watch?v=" + video['id']['videoId'])
        count = count+1
    

Search ID:  {'kind': 'youtube#video', 'videoId': 'PnWJjf1dQJ0'}

Snippet Data: {'publishedAt': '2023-11-25T15:42:18Z', 'channelId': 'UCB_qr75-ydFVKSF9Dmo6izg', 'title': 'Qualifying Highlights | 2023 Abu Dhabi Grand Prix', 'description': 'For one last time this season, the drivers battled it out in Abu Dhabi for the final pole position of 2023! For more F1® videos, visit ...', 'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/PnWJjf1dQJ0/default.jpg', 'width': 120, 'height': 90}, 'medium': {'url': 'https://i.ytimg.com/vi/PnWJjf1dQJ0/mqdefault.jpg', 'width': 320, 'height': 180}, 'high': {'url': 'https://i.ytimg.com/vi/PnWJjf1dQJ0/hqdefault.jpg', 'width': 480, 'height': 360}}, 'channelTitle': 'FORMULA 1', 'liveBroadcastContent': 'none', 'publishTime': '2023-11-25T15:42:18Z'}
1 : https://www.youtube.com/watch?v=PnWJjf1dQJ0
2 : https://www.youtube.com/watch?v=GJMSyW2GYZU
3 : https://www.youtube.com/watch?v=H4XUkc3Q3AU
4 : https://www.youtube.com/watch?v=Q9qSwDxF6YI
5 : https://www.y

## Calculate and Find top 10 most trending videos under that tag

In [78]:
collection_of_video_data = {}
for video in videos:
    if('videoId' in video['id']):
        request = youtube.videos().list(part = 'snippet, statistics', id = video['id']['videoId'])
        response = request.execute()
        collection_of_video_data.append(response)

In [2]:
video_id_to_trend_total = []
for vid_data in collection_of_video_data:
    trendTotal = (vid_data['items'][0]['statistics']['viewCount'] + vid_data['items'][0]['statistics']['likeCount'] + 
                 vid_data['items'][0]['statistics']['commentCount'])
    print(trendTotal)

#     for video in videos:
#         video_id = video['id']['videoId']
#         video_id_to_trend_total[video_id].append(trendTotal)
# print(video_id_to_trend_total)


NameError: name 'collection_of_video_data' is not defined